In [1]:
# NOTE: PLEASE MAKE SURE YOU ARE RUNNING THIS IN A PYTHON3 ENVIRONMENT

import tensorflow as tf
print(tf.__version__)

# This is needed for the iterator over the data
# But not necessary if you have TF 2.0 installed
#!pip install tensorflow==2.0.0-beta0


tf.enable_eager_execution()

# !pip install -q tensorflow-datasets

1.15.0-rc3


In [2]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/0.1.0. Subsequent calls will reuse this data.


In [0]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []


# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s,l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())

  
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)



In [0]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)



In [6]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[1]))
print(training_sentences[1])

<OOV> young iranian women dress as boys and try to get into a world cup <OOV> match between iran and <OOV> when they 're caught they 're penned in an area where the match remains within <OOV> but out of sight the prisoners <OOV> to be let go but rules are rules br br given the <OOV> of its director <OOV> panahi it was <OOV> to discover that <OOV> is a comedy and a frequently hilarious one in 1997 's the mirror panahi presents two versions of iranian <OOV> and leaves the audience to wonder which one is real in 2000 's the circle several iranian women step outside the system their <OOV> are different but they all end up
b'Several young Iranian women dress as boys and try to get into a World Cup qualifying match between Iran and Bahrain. When they\'re caught, they\'re penned in an area where the match remains within earshot, but out of sight. The prisoners plead to be let go, but rules are rules.<br /><br />Given the pedigree of its director, Jafar Panahi, it was disarming to discover tha

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [8]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


25000/25000 [==============================] - 11s 438us/sample - loss: 0.4827 - acc: 0.7597 - val_loss: 0.3531 - val_acc: 0.8452
Epoch 2/10
25000/25000 [==============================] - 9s 375us/sample - loss: 0.2431 - acc: 0.9048 - val_loss: 0.3747 - val_acc: 0.8364
Epoch 3/10
25000/25000 [==============================] - 9s 373us/sample - loss: 0.1011 - acc: 0.9730 - val_loss: 0.4442 - val_acc: 0.8282
Epoch 4/10
25000/25000 [==============================] - 9s 371us/sample - loss: 0.0295 - acc: 0.9960 - val_loss: 0.5147 - val_acc: 0.8300
Epoch 5/10
25000/25000 [==============================] - 9s 372us/sample - loss: 0.0119 - acc: 0.9986 - val_loss: 0.5920 - val_acc: 0.8244
Epoch 6/10
25000/25000 [==============================] - 9s 377us/sample - loss: 0.0056 - acc: 0.9992 - val_loss: 0.6419 - val_acc: 0.8221
Epoch 7/10
25000/25000 [==============================] - 9s 376us/sample - loss: 0.0022 - acc: 0.9998 - val_loss: 0.7173 - val_acc: 0.8215
Epoch 8/10
25000/25000 [======

In [9]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [0]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [0]:


try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

In [13]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences(sentence)
print(sequence)

[[11], [], [1430], [968], [4], [1540], [1540], [4740], [], [790], [2011], [11], [2928], [2192], [], [790], [2011], [11], [579], [], [11], [579], [], [4], [1780], [4], [4505], [11], [2928], [1276], [], [], [2011], [1005], [2928], [968], [579], [790], []]
